In [13]:
import numpy as np
import tensorflow as tf

def Entry_sep_block(x, filters,num):
    x_shortcut = x
    if num != 1:
        x = tf.keras.layers.Activation('relu')(x)
    else:
        x = x
    
    x = tf.keras.layers.SeparableConv2D(filters=filters,
                                       kernel_size=3,
                                       strides=1,
                                       padding='same',
                                       use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.SeparableConv2D(filters=filters,
                                       kernel_size=3,
                                       strides=1,
                                       padding='same',
                                       use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same')(x)
    
    x_shortcut = tf.keras.layers.Conv2D(filters=filters,
                                       kernel_size=1,
                                       strides=2,
                                       padding='same',
                                       use_bias=False)(x_shortcut)
    x_shortcut = tf.keras.layers.BatchNormalization()(x_shortcut)
    
    x = tf.keras.layers.Add()([x,x_shortcut])
    return x

def Entry_flow(x):
    x = tf.keras.layers.Conv2D(filters=32,
                                       kernel_size=3,
                                       strides=2,
                                       padding='same',
                                       use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)    
    
    x = tf.keras.layers.Conv2D(filters=64,
                                       kernel_size=3,
                                       strides=1,
                                       padding='same',
                                       use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)    
    
    x = Entry_sep_block(x,128,1)
    x = Entry_sep_block(x,256,2)
    x = Entry_sep_block(x,728,3)
    return x

def Middle_flow(x,filters):
    for _ in range(8):
        x_shortcut = x
        x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.SeparableConv2D(filters=filters,
                                           kernel_size=3,
                                           strides=1,
                                           padding='same',
                                           use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.SeparableConv2D(filters=filters,
                                           kernel_size=3,
                                           strides=1,
                                           padding='same',
                                           use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.SeparableConv2D(filters=filters,
                                           kernel_size=3,
                                           strides=1,
                                           padding='same',
                                           use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Add()([x,x_shortcut])
    return x

def Exit_flow(x,filters):
    f1, f2, f3, f4 = filters
    x_shortcut = x
    
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.SeparableConv2D(filters=f1,
                                           kernel_size=3,
                                           strides=1,
                                           padding='same',
                                           use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.SeparableConv2D(filters=f2,
                                           kernel_size=3,
                                           strides=1,
                                           padding='same',
                                           use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same')(x)
    x_shortcut = tf.keras.layers.Conv2D(filters=f2,
                                       kernel_size=1,
                                       strides=2,
                                       padding='same',
                                       use_bias=False)(x_shortcut)
    x_shortcut = tf.keras.layers.BatchNormalization()(x_shortcut)
    x = tf.keras.layers.Add()([x,x_shortcut])
    
    x = tf.keras.layers.SeparableConv2D(filters=f3,
                                           kernel_size=3,
                                           strides=1,
                                           padding='same',
                                           use_bias=False)(x)    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.SeparableConv2D(filters=f4,
                                           kernel_size=3,
                                           strides=1,
                                           padding='same',
                                           use_bias=False)(x)    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    
    x = tf.keras.layers.Dense(units = 1000, activation='softmax')(x)
    
    return x

def Xception():
    x_input = tf.keras.layers.Input((299,299,3))
    
    x = Entry_flow(x_input)
    x = Middle_flow(x,728)
    x = Exit_flow(x,[728,1024,1536,2048])
    
    model = tf.keras.models.Model(inputs=x_input,outputs=x,name='Xception')

    return model

model = Xception()
model.summary()

Model: "Xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 150, 150, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_96 (BatchNo (None, 150, 150, 32) 128         conv2d_20[0][0]                  
__________________________________________________________________________________________________
activation_76 (Activation)      (None, 150, 150, 32) 0           batch_normalization_96[0][0]     
___________________________________________________________________________________________